In [1]:
from langchain.llms import HuggingFaceEndpoint
import os

# Correct parameter name
endpoint_url = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.1-8B"

llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,  
    huggingfacehub_api_token=os.environ['HF_TOKEN'],
    max_new_tokens=500,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)


C:\Users\bhave\AppData\Local\Temp\ipykernel_10324\1089803420.py:7: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(
c:\Users\bhave\anaconda3\envs\serach\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [6]:
from duckduckgo_search import DDGS
from langchain.tools import BaseTool


class WebSearch(BaseTool):
    name: str = "Search Engine"
    description: str = "Use this search Engine to answer the user query based on Web search"
     
     
    def _run(self,query: str):
        results = DDGS().text(query,max_results=10)
        all_results = " "
        for  i in range(len(results)):
            search_results = results[i]['title'] + results[i]['body']
            all_results += search_results + "\n\n"
            
        return all_results
    
web_tool = WebSearch()
            
            
        
         
    



In [7]:
from langchain.agents import AgentType,initialize_agent

tools =[web_tool]

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent_type = AgentType.REACT_DOCSTORE,
    max_iterations = 5,
    verbose = True
    #search_engine_kwargs = {'search_engine': 'DuckDuckGoSearch'}
)
         


In [9]:
agent.invoke('what is todays date')



> Entering new AgentExecutor chain...


c:\Users\bhave\anaconda3\envs\serach\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


ValueError: Task unknown has no recommended model. Please specify a model explicitly. Visit https://huggingface.co/tasks for more info.

In [11]:
import os
from huggingface_hub import InferenceClient
from langchain.tools import BaseTool
from duckduckgo_search import DDGS
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatGroq
from langchain.chat_models import groq


# Initialize the Hugging Face inference client
client = InferenceClient(model="meta-llama/Llama-3.1-8B")

def query_llm(prompt):
    response = client.chat_completion(messages=[{"role": "user", "content": prompt}])
    return response['choices'][0]['message']['content']

# Define Web Search Tool
class WebSearch(BaseTool):
    name: str = "Search Engine"
    description: str = "Use this search engine to answer queries based on web search results."
     
    def _run(self, query: str):
        results = DDGS().text(query, max_results=10)
        all_results = "\n\n".join(f"{r['title']}: {r['body']}" for r in results)
        return all_results
    
web_tool = WebSearch()

# Initialize LangChain agent
tools = [web_tool]

agent = initialize_agent(
    tools=tools,
    llm= ChatGroq(model_name="gpt-4"),  # Use OpenAI model for better tool execution
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=5,
    verbose=True
)

# Run the agent
response = agent.invoke("What is today's date?")
print(response)


ImportError: cannot import name 'ChatGroq' from 'langchain.chat_models' (c:\Users\bhave\anaconda3\envs\serach\Lib\site-packages\langchain\chat_models\__init__.py)